1. Creating a Delta Table

In [0]:
# Create a Spark session with Delta support
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("DeltaDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Create a sample DataFrame
data = [("James", 34), ("Michael", 45), ("Robert", 38)]
df = spark.createDataFrame(data, ["name", "age"])

# Write as Delta table
df.write.format("delta").save("/tmp/delta/people")

2. Reading a Delta Table

In [0]:
# Read the Delta table
delta_df = spark.read.format("delta").load("/tmp/delta/people")
delta_df.show()

+-------+---+
|   name|age|
+-------+---+
|  James| 34|
|Michael| 45|
| Robert| 38|
+-------+---+



3. Updating Data (Time Travel)

In [0]:
# Update the table
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, "/tmp/delta/people")

# Update age where name = James
deltaTable.update(
    condition = "name = 'James'",
    set = { "age": "35" }
)

# Show version history
deltaTable.history().show()

+-------+-------------------+---------------+--------------------+---------+--------------------+----+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|         userId|            userName|operation| operationParameters| job|notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+---------------+--------------------+---------+--------------------+----+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      1|2025-08-14 04:31:37|144757880474156|azuser4024_mml.lo...|   UPDATE|{predicate -> ["(...|NULL|    NULL|0814-033924-qe48w...|          0|WriteSerializable|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      0|2025-08-14 04:31:09|144757880474156|azuser4024_mml.lo...|    WR

4. Time Travel Demonstration

In [0]:
# Read version 0 (original data)
original_df = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load("/tmp/delta/people")
original_df.show()

+-------+---+
|   name|age|
+-------+---+
|  James| 34|
|Michael| 45|
| Robert| 38|
+-------+---+

